## Patents Trend: Data Exploration and Cleaning

This notebook contains the process to prepare a suitable dataset to be used data visualization projects, such as the Tableau project you can find [here](https://public.tableau.com/views/U_S_PatentsDashboardFinal/Dash0_1Home?:language=en-GB&:display_count=n&:origin=viz_share_link). The main interest of the project is to study: 
1) Geographical distribution of patents assignees, and how this might have changed over the last decades.  
2) Historical trends in patents granting

The dataset for the second point is explored and refined in [this]() separate notebook.
As for the first point, I am going to rely on data made available by the USPTO (United States Patent and Trademark Office) about patents assignments. The full dataset can be found [here](https://www.uspto.gov/ip-policy/economic-research/research-datasets/patent-assignment-dataset), including the [database schema](https://www.uspto.gov/sites/default/files/documents/pat_assign_dataset_schema.pdf). I will use 3 out of the 6 tables available: *assignment*, *documentid* and *assignee*."*"

"*" For a complete description of the database and how it was obtained, see Graham, SJH, Marco, AC, Myers, AF. Patent transactions in the marketplace: Lessons from the USPTO Patent Assignment Dataset. J Econ Manage Strat. 2018; 27: 343– 371. [DOI](https://doi.org/10.1111/jems.12262)

In [20]:
import pandas as pd
import numpy as np

I am going to import, explore and do a first cleaning round for each of the three tables we are interested in. The tables contain between 9 and 11 million rows, plus several columns, so this round will be important to then merge refined tables containing only the necessary information for the project. 

*Note*: For convenience, this notebook relies on csv downloaded locally. Due to the very large size of these csv files, they could not be uploaded in this Github repository. In case you want to access the file, you can do so via the USPTO website. As a reference, you can find the code needed to automatically extract the files from the USPTO portal in the notebook called *extractor_zip_csv*.

## **First table Exploration: Assignment**
Assignment contains the primary key rf_id used to uniquely identify each patent assignment, plus several information about it.

In [21]:
assignment = pd.read_csv('../csv/assignment.csv')
assignment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9547604 entries, 0 to 9547603
Data columns (total 14 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   rf_id           int64 
 1   file_id         int64 
 2   cname           object
 3   caddress_1      object
 4   caddress_2      object
 5   caddress_3      object
 6   caddress_4      object
 7   reel_no         int64 
 8   frame_no        int64 
 9   convey_text     object
 10  record_dt       object
 11  last_update_dt  object
 12  page_count      int64 
 13  purge_in        object
dtypes: int64(5), object(9)
memory usage: 1019.8+ MB


Let's drop all those columns containing non-relevant information, and let's check how many NaN values we have in each column remaining.

In [22]:
assignment = assignment.drop(columns=["cname","caddress_1","caddress_2","caddress_3","caddress_4","reel_no","frame_no", "purge_in", "convey_text", "last_update_dt", "record_dt", "page_count"])

In [23]:
assignment.isna().sum()

rf_id      0
file_id    0
dtype: int64

## **Second table Exploration: Assignee**
The Assignee table contains several information about the assignees of a given patent, including their names, and geospatial information. I am mostly interested in their City and Country, and possibly in Postcode and State. Any other column will be deleted.

In [24]:
assignee = pd.read_csv('../csv/assignee.csv')
assignee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9901731 entries, 0 to 9901730
Data columns (total 8 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   rf_id         int64 
 1   ee_name       object
 2   ee_address_1  object
 3   ee_address_2  object
 4   ee_city       object
 5   ee_state      object
 6   ee_postcode   object
 7   ee_country    object
dtypes: int64(1), object(7)
memory usage: 604.4+ MB


In [25]:
assignee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9901731 entries, 0 to 9901730
Data columns (total 8 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   rf_id         int64 
 1   ee_name       object
 2   ee_address_1  object
 3   ee_address_2  object
 4   ee_city       object
 5   ee_state      object
 6   ee_postcode   object
 7   ee_country    object
dtypes: int64(1), object(7)
memory usage: 604.4+ MB


In [26]:
#As before, remove unnecessary columns
assignee = assignee.drop(columns=["ee_address_2","ee_name","ee_address_1"])

In [27]:
#Check for NaN values
assignee.isna().sum()

rf_id                0
ee_city         247310
ee_state       5046695
ee_postcode    2303323
ee_country     4991914
dtype: int64

In [28]:
#To have a better idea of why we might have null values, let us see the first 20 rows of the df.
assignee.iloc[0:20]

,rf_id,ee_city,ee_state,ee_postcode,ee_country
0,12800340,NaN,NaN,NaN,NaN
1,36250888,NaN,NaN,NaN,NOT PROVIDED
2,36340874,LARGO,FLORIDA,33540,NaN
3,36340875,LARGO,FLORIDA,33540,NaN
4,36920537,COOKEVILLE,TENNESSEE,38501,NaN
5,36960452,NaN,NaN,NaN,NOT PROVIDED
6,37290735,CUPERTINO,CALIFORNIA,95014,NaN
7,37370606,"VOROSMARTY U. 67,",NaN,NaN,HUNGARY
8,37410900,NaN,NaN,NaN,NOT PROVIDED
9,37650757,5 - 20123 MILAN,NaN,NaN,ITALY


For my goals, we need to have *ee_city* and *ee_country* as cleaned as possible. This entails two steps: first, we need to strip away from ee_city any character that is not an alphabet letter (e.g. numerical digits, commas, full stops etc.), and strip any trailing space. 

In [29]:
#Clean ee_city from unwanted characters using regular expressions.
assignee['ee_city'] = assignee['ee_city'].str.replace(r'[^a-zA-Z\s]', '', regex=True).str.strip()

#Check that it worked as expected, comparing some rows that needed cleaning.
assignee.iloc[6:16]

,rf_id,ee_city,ee_state,ee_postcode,ee_country
6,37290735,CUPERTINO,CALIFORNIA,95014,NaN
7,37370606,VOROSMARTY U,NaN,NaN,HUNGARY
8,37410900,NaN,NaN,NaN,NOT PROVIDED
9,37650757,MILAN,NaN,NaN,ITALY
10,37880668,NaN,NaN,NaN,NOT PROVIDED
11,37880670,NaN,NaN,NaN,NOT PROVIDED
12,37880673,HAMBURG,NaN,D-2000,GERMANY
13,37880675,NaN,NaN,NaN,NaN
14,37880676,NaN,NaN,NaN,NOT PROVIDED
15,37880677,MARIETTA,GEORGIA,NaN,NaN


Secondly, we need to refine ee_country by replacing "NaN" values with "USA" values for all those rows where there is a value under "ee_state" (this is because, in the dataset, any row containing an American state was left blank with regard to its overall country). I am going to define a custom function **update_country** to do so.

In [30]:
def update_country(row):
    """Check the ee_country column and the ee_state column for each row. 
    If the former is NaN, while the former is not NaN, it means that the state is a US state. 
    Hence, overwrite 'USA' under the ee_country column. Otherwise, keep whatever value is in there."""
    if pd.isna(row['ee_country']) and not pd.isna(row['ee_state']):
        return 'USA'
    else:
        return row['ee_country']

In [31]:
#Assign the result of the function to replace the value of the ee_country column, 
#and check the first 5 rows to make sure it worked. The first row should have still NaN for the country,
#while the third, fourth, and fifth row should now display 'USA'. 
assignee["ee_country"] = assignee.apply(update_country, axis=1)
assignee.iloc[0:5]

,rf_id,ee_city,ee_state,ee_postcode,ee_country
0,12800340,NaN,NaN,NaN,NaN
1,36250888,NaN,NaN,NaN,NOT PROVIDED
2,36340874,LARGO,FLORIDA,33540,USA
3,36340875,LARGO,FLORIDA,33540,USA
4,36920537,COOKEVILLE,TENNESSEE,38501,USA


Once this is done, we can drop any row containing a NaN value in the **ee_country** column, since it is an essential information we want to work with.

In [32]:
assignee = assignee.dropna(subset="ee_country")

## **Third table Exploration: DocumentId**
DocumentId contains information about several steps from the application to the actual grant of a patent. In this case, I am mostly interested in retrieving information about the application and the grant date (to estimate how long on average it takes for a patent to be granted), so the other columns will be deleted. 

In [33]:
documentid = pd.read_csv("../csv/documentid.csv")
documentid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16254049 entries, 0 to 16254048
Data columns (total 12 columns):
 #   Column         Dtype  
---  ------         -----  
 0   rf_id          int64  
 1   title          object 
 2   lang           object 
 3   appno_doc_num  float64
 4   appno_date     object 
 5   appno_country  object 
 6   pgpub_doc_num  float64
 7   pgpub_date     object 
 8   pgpub_country  object 
 9   grant_doc_num  object 
 10  grant_date     object 
 11  grant_country  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 1.5+ GB


In [34]:
#Drop unnecessary columns
documentid = documentid.drop(columns=["title","lang","appno_doc_num","pgpub_doc_num","pgpub_date","pgpub_country","grant_doc_num", "grant_country", "appno_country"])

Let us drop the rows in documentid having null values, giving that we are interested in documents with full data available as of grant dates, and drop any duplicates keeping only the last duplicated occurrence (which corresponds to the assignment with the most recent date). 

In [35]:
#Drop rows containing NaN
documentid = documentid.dropna()

In [36]:
documentid

,rf_id,appno_date,grant_date
0,12800340,1994-10-20,1999-01-12
1,36250888,1977-11-28,1979-04-17
3,36250888,1975-05-05,1978-12-26
7,36250888,1983-02-16,1985-02-05
14,36340874,1979-03-02,1981-06-16
...,...,...,...
16254034,591080104,2018-07-13,2020-12-01
16254035,591080104,2019-01-31,2020-04-07
16254036,591080104,2019-06-14,2020-12-01
16254038,591080104,2019-07-17,2021-03-30


In [37]:
#Convert the grant_date and appno_date into datetime type columns, using a lambda function.
documentid[["grant_date", "appno_date"]] = documentid[["grant_date", "appno_date"]].apply(lambda x: pd.to_datetime(x, format="%Y-%m-%d"))
documentid

,rf_id,appno_date,grant_date
0,12800340,1994-10-20,1999-01-12
1,36250888,1977-11-28,1979-04-17
3,36250888,1975-05-05,1978-12-26
7,36250888,1983-02-16,1985-02-05
14,36340874,1979-03-02,1981-06-16
...,...,...,...
16254034,591080104,2018-07-13,2020-12-01
16254035,591080104,2019-01-31,2020-04-07
16254036,591080104,2019-06-14,2020-12-01
16254038,591080104,2019-07-17,2021-03-30


In [38]:
documentid.describe()

,rf_id,appno_date,grant_date
count,1.330221e+07,13302208,13302208
mean,2.715046e+08,2004-06-30 23:34:05.447852800,2007-03-08 09:31:24.824652800
min,1.280034e+07,1944-07-22 00:00:00,1946-11-05 00:00:00
25%,1.374601e+08,1998-04-03 00:00:00,2000-08-01 00:00:00
50%,2.611404e+08,2005-06-20 00:00:00,2009-01-13 00:00:00
75%,3.919102e+08,2012-08-02 00:00:00,2015-05-12 00:00:00
max,5.910806e+08,2038-09-23 00:00:00,2038-02-08 00:00:00
std,1.519624e+08,NaN,NaN


Clearly, there must be mistakes in the dataset, since no date can be past 2022. I will hence filter out any row containing "invalid" years either in the appno_date or in the grant_date, making sure that all entries have a date before the 31st December 2022.

In [39]:
#Filter the dataframe to avoid invalid years
documentid = documentid[(documentid["appno_date"]<'2022-12-31') & (documentid["grant_date"]<'2022-12-31')]

In [40]:
#Extract year from appno_date and grant_date, and store it into two new columns. 
documentid.loc[:, "appno_year"] = documentid["appno_date"].dt.year

documentid.loc[:,"grant_year"] = documentid["grant_date"].dt.year

C:\Users\matte\AppData\Local\Temp\ipykernel_24280\215963960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documentid.loc[:, "appno_year"] = documentid["appno_date"].dt.year
C:\Users\matte\AppData\Local\Temp\ipykernel_24280\215963960.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documentid.loc[:,"grant_year"] = documentid["grant_date"].dt.year


In [41]:
#Sort rows in ascending order by grant_date and appno_date, so that we can then drop all duplicates but the last. 
documentid = documentid.sort_values(by=["grant_date","appno_date"])
documentid

,rf_id,appno_date,grant_date,appno_year,grant_year
196061,40870328,1973-11-09,1946-11-05,1973,1946
193114,40820607,1970-06-01,1947-06-22,1970,1947
314302,42590685,1948-01-12,1949-04-05,1948,1949
546866,46050690,1948-01-12,1949-04-05,1948,1949
652592,47470743,1948-01-12,1949-04-05,1948,1949
...,...,...,...,...,...
16245965,590000881,2020-12-22,2022-03-15,2020,2022
16251368,590890497,2021-01-15,2022-03-15,2021,2022
16246726,590130938,2021-04-15,2022-03-15,2021,2022
16246442,590080899,2021-04-28,2022-03-15,2021,2022


In [56]:
#Drop all duplicates with regard to the rf_id column, except the last occurrence of each duplicate
documentid = documentid.drop_duplicates(subset='rf_id', keep='last')
documentid

,rf_id,appno_date,grant_date,appno_year,grant_year
196061,40870328,1973-11-09,1946-11-05,1973,1946
426653,44230097,1956-06-25,1958-03-11,1956,1958
416999,44050575,1971-12-07,1958-03-11,1971,1958
89717,39320635,1958-09-22,1959-07-07,1958,1959
10633,38060936,1957-10-04,1959-09-22,1957,1959
...,...,...,...,...,...
16245965,590000881,2020-12-22,2022-03-15,2020,2022
16251368,590890497,2021-01-15,2022-03-15,2021,2022
16246726,590130938,2021-04-15,2022-03-15,2021,2022
16246442,590080899,2021-04-28,2022-03-15,2021,2022


### Creating the Unified Table
Now that all the three tables of interest have been explored and refined, we can proceed merging them into a single dataframe. We are going to merge them using **inner joins** with **rf_id** as common key, so that only rows with matching rf_id are combined.

In [57]:
#First we merge assignment and assignee, and we then use the resulting df to merge also documentid.
first_merge = pd.merge(assignment, assignee, 'inner',on=['rf_id','rf_id'])
merged_df = pd.merge(first_merge, documentid, 'inner', on=['rf_id','rf_id'])

In [58]:
merged_df.iloc[:].isnull().sum()

rf_id                0
file_id              0
ee_city         107750
ee_state       3664760
ee_postcode    1824798
ee_country           0
appno_date           0
grant_date           0
appno_year           0
grant_year           0
dtype: int64

As expected, the columns we wanted to have 0 null values (i.e. country, and the appl/grant dates) actually have 0 null values. Now, to avoid problems later on, let's strip any trailing spaces from the country column.

In [59]:
merged_df.loc[:,'ee_country'] = merged_df["ee_country"].str.strip()

From the head of the dataframe we can see that the operations worked as expected, and the df now looks like what we want.

In [60]:
merged_df.head()

,rf_id,file_id,ee_city,ee_state,ee_postcode,ee_country,appno_date,grant_date,appno_year,grant_year
0,36250888,1,NaN,NaN,NaN,NOT PROVIDED,1983-02-16,1985-02-05,1983,1985
1,36340874,1,LARGO,FLORIDA,33540,USA,1979-03-02,1981-06-16,1979,1981
2,36340875,1,LARGO,FLORIDA,33540,USA,1979-03-02,1983-04-26,1979,1983
3,36960452,1,NaN,NaN,NaN,NOT PROVIDED,1977-08-04,1981-04-14,1977,1981
4,37290735,1,CUPERTINO,CALIFORNIA,95014,USA,1980-01-28,1982-04-06,1980,1982


In order to draw correlation analyses involving the **ee_country**, it will be handy to have such column identified as a categorical variable, and to have an associated column with the same code every time the same country appears in a row. We can do so by converting the column into a *category* type, and then using pandas **cat.codes**. 

In [61]:
merged_df["ee_country"] = merged_df["ee_country"].astype("category")
merged_df.loc[:,"country_numerized"] = merged_df["ee_country"].cat.codes

merged_df.head()

,rf_id,file_id,ee_city,ee_state,ee_postcode,ee_country,appno_date,grant_date,appno_year,grant_year,country_numerized
0,36250888,1,NaN,NaN,NaN,NOT PROVIDED,1983-02-16,1985-02-05,1983,1985,168
1,36340874,1,LARGO,FLORIDA,33540,USA,1979-03-02,1981-06-16,1979,1981,241
2,36340875,1,LARGO,FLORIDA,33540,USA,1979-03-02,1983-04-26,1979,1983,241
3,36960452,1,NaN,NaN,NaN,NOT PROVIDED,1977-08-04,1981-04-14,1977,1981,168
4,37290735,1,CUPERTINO,CALIFORNIA,95014,USA,1980-01-28,1982-04-06,1980,1982,241


Finally, we can use the cleaned dataframe to produce a new csv that we will use for future analysis. 

In [62]:
merged_df.to_csv("../csv//CleanedDatasetPatents.csv",index=False,header=["id","file_id","applicant_city","applicant_state","postcode","country","date_of_application","date_of_grant","application_year","grant_year","country_numerized"])